<a href="https://colab.research.google.com/github/IImbryk/NMA_MindBigData/blob/work/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from google.colab import drive

In [5]:
drive.mount('/content/drive', force_remount=True)
path_dir = '/content/drive/MyDrive/Data Brain MNIST'

Mounted at /content/drive


In [6]:
file_name =  ['/MindBigData-EP-v1.0/EP1.01.txt',
              '/MindBigData-IN-v1.06/IN.txt',
              '/MindBigData-MU-v1.0/MU.txt',
              '/MindBigData-MW-v1.0/MW.txt']

In [7]:
#load data
data_2 = pd.read_csv(path_dir+file_name[2], sep="\t", header=None, nrows=999)
data = data_2.copy(deep=True)

In [8]:
data

,0,1,2,3,4,5,6
0,978111,132669,MU,TP9,6,459,"475,474,477,486,486,476,479,483,489,483,482,48..."
1,978112,132669,MU,FP1,6,459,"468,487,493,493,498,493,491,490,492,487,483,48..."
2,978113,132669,MU,FP2,6,459,"482,475,490,500,485,470,470,482,490,484,478,48..."
3,978114,132669,MU,TP10,6,459,"470,470,478,489,487,475,469,478,488,483,474,48..."
4,978115,132670,MU,TP9,7,493,"506,499,495,491,492,507,496,500,498,496,499,50..."
...,...,...,...,...,...,...,...
994,979105,132917,MU,FP2,9,493,"499,491,489,492,493,491,490,484,491,496,494,49..."
995,979106,132917,MU,TP10,9,493,"514,509,493,495,511,512,492,498,511,518,506,49..."
996,979107,132918,MU,TP9,6,459,"528,521,527,534,531,517,523,532,522,538,521,52..."
997,979108,132918,MU,FP1,6,459,"516,529,531,525,524,520,521,533,524,523,529,52..."


In [9]:
data = data.rename(columns={0: 'id', 1: 'event',
                        2: 'device', 3:'channel',
                        4: 'code', 5: 'size', 6:'data'})

In [10]:
data

,id,event,device,channel,code,size,data
0,978111,132669,MU,TP9,6,459,"475,474,477,486,486,476,479,483,489,483,482,48..."
1,978112,132669,MU,FP1,6,459,"468,487,493,493,498,493,491,490,492,487,483,48..."
2,978113,132669,MU,FP2,6,459,"482,475,490,500,485,470,470,482,490,484,478,48..."
3,978114,132669,MU,TP10,6,459,"470,470,478,489,487,475,469,478,488,483,474,48..."
4,978115,132670,MU,TP9,7,493,"506,499,495,491,492,507,496,500,498,496,499,50..."
...,...,...,...,...,...,...,...
994,979105,132917,MU,FP2,9,493,"499,491,489,492,493,491,490,484,491,496,494,49..."
995,979106,132917,MU,TP10,9,493,"514,509,493,495,511,512,492,498,511,518,506,49..."
996,979107,132918,MU,TP9,6,459,"528,521,527,534,531,517,523,532,522,538,521,52..."
997,979108,132918,MU,FP1,6,459,"516,529,531,525,524,520,521,533,524,523,529,52..."


In [11]:
for i in range(0, len(data)):
  data['data'][i]=list(map(int, data['data'][i].split(',')))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


In [13]:
df_train = pd.DataFrame({'data':[], 'code':[]})

In [14]:
for x in range(len(data['size'])):
  if data['size'][x] == 459:
    df_train = df_train.append({'data': data['data'][x], 'code':data['code'][x].astype(int)}, ignore_index=True)

In [15]:
df_train

,data,code
0,"[475, 474, 477, 486, 486, 476, 479, 483, 489, ...",6.0
1,"[468, 487, 493, 493, 498, 493, 491, 490, 492, ...",6.0
2,"[482, 475, 490, 500, 485, 470, 470, 482, 490, ...",6.0
3,"[470, 470, 478, 489, 487, 475, 469, 478, 488, ...",6.0
4,"[520, 533, 528, 517, 522, 534, 534, 531, 522, ...",6.0
...,...,...
274,"[503, 506, 507, 506, 499, 493, 505, 507, 511, ...",9.0
275,"[529, 519, 521, 522, 512, 503, 517, 523, 515, ...",9.0
276,"[528, 521, 527, 534, 531, 517, 523, 532, 522, ...",6.0
277,"[516, 529, 531, 525, 524, 520, 521, 533, 524, ...",6.0


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [39]:
import torch
import torch.nn as nn

time_steps = 279
batch_size = 1
in_size = 459
classes_no = 1
epochs = 5

model = nn.LSTM(in_size, classes_no, 2)
# input_seq = Variable(torch.randn(time_steps, batch_size, in_size))
input_seq =torch.Tensor(df_train['data']).unsqueeze(0)
output_seq, _ = model(input_seq)
last_output = output_seq[-1]

loss = nn.MSELoss()
target = torch.Tensor(df_train['code'].astype(int))
err = loss(last_output, target)
err.backward()

model.train()
for i in range(epochs):
    # h = model.init_hidden(batch_size)
    for inputs, labels in (torch.Tensor(df_train['data']).unsqueeze(0), target):
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if counter%print_every == 0:
            val_h = model.init_hidden(batch_size)
            val_losses = []
            model.eval()
            for inp, lab in val_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = criterion(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
                
            model.train()
            print("Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './state_dict.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([279])) that is different to the input size (torch.Size([279, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


ValueError: ignored

In [ ]:
target

In [ ]:
last_output[0]

In [ ]:
last_output.size()

In [3]:
last_output.squeeze(1).size()

NameError: ignored

In [ ]:
last_output